# Dataset Cleaning

This notebook is designed to perform **data cleaning** on the dataset used for the **Café X Product Performance Analysis** project and prepare it for analysis.

### To-Do List Based on Data Validation Findings

**1. Remove Irrelevant an Unusable Columns**
- Drop columns that do not contribute to the project objectives.

**2. Convert Column Headers to Snake Case**
- Standardize all column names to lowercase with underscores for consistency and easier coding.

**3. Handle Missing and Invalid Values**
- Supplement missing values where possible.
- Standardize missing-value representation.



In [195]:
#Import necessary library
import pandas as pd
import numpy as np

In [196]:
#Load the dataset

menu = pd.read_csv("<path-to-file>/dirty_cafe_sales.csv")

In [197]:
# Check
menu.head(3)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2,4,Credit Card,Takeaway,08/09/2023
1,TXN_4977031,Cake,4,3,12,Cash,In-store,16/05/2023
2,TXN_4271903,Cookie,4,1,ERROR,Credit Card,In-store,19/07/2023


In [198]:
# Make a copy to avoid modifying original directly
menu_clean = menu.copy()

In [199]:
# Check
menu_clean.head(3)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2,4,Credit Card,Takeaway,08/09/2023
1,TXN_4977031,Cake,4,3,12,Cash,In-store,16/05/2023
2,TXN_4271903,Cookie,4,1,ERROR,Credit Card,In-store,19/07/2023


## Drop Unnecessary and Unusable Columns

Columns **Payment Method** and **Location** will be dropped because they contain too many vague or invalid values and do not contribute significantly to achieving the objectives of the project.

In [200]:
# Drop columns from the DataFrame
menu_clean = menu_clean.drop(columns=['Payment Method', 'Location'])

In [201]:
# Check
menu_clean.head(3)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Transaction Date
0,TXN_1961373,Coffee,2,2,4,08/09/2023
1,TXN_4977031,Cake,4,3,12,16/05/2023
2,TXN_4271903,Cookie,4,1,ERROR,19/07/2023


## Convert Headers to Snake Case

To improve readability and avoid potential issues in coding or analysis, all column headers will be converted to **snake_case** (e.g., `price_per_unit`).

In [202]:
# Standardize column names
menu_clean.columns = menu_clean.columns.str.strip().str.lower().str.replace(' ', '_')

In [203]:
# Check
menu_clean.head(3)

,transaction_id,item,quantity,price_per_unit,total_spent,transaction_date
0,TXN_1961373,Coffee,2,2,4,08/09/2023
1,TXN_4977031,Cake,4,3,12,16/05/2023
2,TXN_4271903,Cookie,4,1,ERROR,19/07/2023


## Impute Missing Values

Some missing values can be reconstructed using other available information as a reference.

### Determine Missing Values for columns *quantity*, *price_per_unit*, and *total_spent*

The columns share a deterministic mathematical relationship:

$$
Quantity \times Price\ per\ Unit = Total\ Spent
$$

Because these fields depend directly on one another, any missing value can be accurately reconstructed as long as the remaining two values are present.


In [204]:
# Convert to numeric, coerce invalid entries to NaN
menu_clean['quantity'] = pd.to_numeric(menu_clean['quantity'], errors='coerce')
menu_clean['price_per_unit'] = pd.to_numeric(menu_clean['price_per_unit'], errors='coerce')
menu_clean['total_spent'] = pd.to_numeric(menu_clean['total_spent'], errors='coerce')

In [205]:
# impute missing values based
for idx, row in menu_clean.iterrows():
    q = row['quantity']
    p = row['price_per_unit']
    t = row['total_spent']
    
    # If quantity is missing
    if pd.isna(q) and pd.notna(p) and pd.notna(t):
        menu_clean.at[idx, 'quantity'] = t / p
    # If price_per_unit is missing
    elif pd.isna(p) and pd.notna(q) and pd.notna(t):
        menu_clean.at[idx, 'price_per_unit'] = t / q
    # If total_spent is missing
    elif pd.isna(t) and pd.notna(q) and pd.notna(p):
        menu_clean.at[idx, 'total_spent'] = q * p

In [206]:
# Check

menu.loc[
    menu['Transaction ID'] == 'TXN_4271903',
    ['Transaction ID', 'Quantity', 'Price Per Unit', 'Total Spent']
]

,Transaction ID,Quantity,Price Per Unit,Total Spent
2,TXN_4271903,4,1,ERROR


In [207]:
# Check

menu_clean.loc[
    menu_clean['transaction_id'] == 'TXN_4271903',
    ['transaction_id', 'quantity', 'price_per_unit', 'total_spent']
]

,transaction_id,quantity,price_per_unit,total_spent
2,TXN_4271903,4.0,1.0,4.0


### Determine Missing Values for Columns *item* and *price_per_unit*

Information regarding the prices of each item is available on the source of the dataset. This information will be used as a reference to supplement missing data.

**Reference Price Table**

| Item      | Price ($) |
|-----------|-----------|
| Coffee    | 2.00      |
| Tea       | 1.50      |
| Sandwich  | 4.00      |
| Salad     | 5.00      |
| Cake      | 3.00      |
| Cookie    | 1.00      |
| Smoothie  | 4.00      |
| Juice     | 3.00      |

In [208]:
# Create reference price table
reference_prices = pd.DataFrame({
    'item': ['Coffee', 'Tea', 'Sandwich', 'Salad', 'Cake', 'Cookie', 'Smoothie', 'Juice'],
    'price_per_unit': [2.00, 1.50, 4.00, 5.00, 3.00, 1.00, 4.00, 3.00]
})

In [209]:
# Check
reference_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item            8 non-null      object 
 1   price_per_unit  8 non-null      float64
dtypes: float64(1), object(1)
memory usage: 260.0+ bytes


In [210]:
#Impute missing/invalid item names based on unique price matches
# Check if a price is invalid (UNKNOWN, ERROR, blank, or NaN)
def is_invalid(val):
    return pd.isna(val) or str(val).strip().upper() in ['UNKNOWN', 'ERROR', '']

# Iterate through rows where item is invalid
for idx, row in menu_clean.iterrows():
    if is_invalid(row['item']):
        price = row['price_per_unit']
        if not is_invalid(price):
            # Find all items in reference table with this price
            matches = reference_prices[reference_prices['price_per_unit'] == price]
            # Only fill if price is unique in reference table
            if len(matches) == 1:
                menu_clean.at[idx, 'item'] = matches['item'].values[0]
            # Else: skip since multiple items share this price


In [211]:
# Check

menu.loc[
    menu['Transaction ID'] == 'TXN_8927252',
    ['Transaction ID', 'Item', 'Price Per Unit']
]

,Transaction ID,Item,Price Per Unit
31,TXN_8927252,UNKNOWN,1


In [212]:
# Check

menu_clean.loc[
    menu_clean['transaction_id'] == 'TXN_8927252',
    ['transaction_id', 'item', 'price_per_unit']
]

,transaction_id,item,price_per_unit
31,TXN_8927252,Cookie,1.0


In [213]:
#Impute missing price_per_unit that was not supplemented using deterministic relationship
# Check if item is invalid
def is_invalid_item(val):
    return pd.isna(val) or str(val).strip().upper() in ['UNKNOWN', 'ERROR']

# Impute missing price_per_unit using reference table
for idx, row in menu_clean.iterrows():
    if pd.isna(row['price_per_unit']):
        item = row['item']
        # Only fill if item is valid
        if not is_invalid_item(item):
            match = reference_prices[reference_prices['item'] == item]
            if not match.empty:
                menu_clean.at[idx, 'price_per_unit'] = match['price_per_unit'].values[0]


In [214]:
# Check

menu.loc[
    menu['Transaction ID'] == 'TXN_4987129',
    ['Transaction ID', 'Item', 'Price Per Unit']
]

,Transaction ID,Item,Price Per Unit
65,TXN_4987129,Sandwich,NaN


In [215]:
# Check

menu_clean.loc[
    menu_clean['transaction_id'] == 'TXN_4987129',
    ['transaction_id', 'item', 'price_per_unit']
]

,transaction_id,item,price_per_unit
65,TXN_4987129,Sandwich,4.0


In [216]:
# Rerun logic to impute values for quantity and total_spent columns
for idx, row in menu_clean.iterrows():
    q = row['quantity']
    p = row['price_per_unit']
    t = row['total_spent']
    
    # If quantity is missing
    if pd.isna(q) and pd.notna(p) and pd.notna(t):
        menu_clean.at[idx, 'quantity'] = t / p
    # If total_spent is missing
    elif pd.isna(t) and pd.notna(q) and pd.notna(p):
        menu_clean.at[idx, 'total_spent'] = q * p

In [217]:
# Check

menu.loc[
    menu['Transaction ID'] == 'TXN_4987129',
    ['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent']
]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent
65,TXN_4987129,Sandwich,3,NaN,NaN


In [218]:
# Check

menu_clean.loc[
    menu_clean['transaction_id'] == 'TXN_4987129',
    ['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent']
]

,transaction_id,item,quantity,price_per_unit,total_spent
65,TXN_4987129,Sandwich,3.0,4.0,12.0


In [219]:
# This is the code used to look for specific columns to confirm if the imputation worked correctly

# Check if item is invalid
def is_invalid_item(val):
    return pd.isna(val) or str(val).strip().upper() in ['UNKNOWN', 'ERROR']

# Filter rows where:
# - item is UNKNOWN, ERROR, or NaN
# - quantity, price_per_unit, or total_spent is NaN
invalid_rows = menu_clean[
    ##menu_clean['item'].apply(is_invalid_item) |
    menu_clean['quantity'].isna() |
    menu_clean['price_per_unit'].isna() |
    menu_clean['total_spent'].isna()
]

# Display the filtered rows
invalid_rows


,transaction_id,item,quantity,price_per_unit,total_spent,transaction_date
236,TXN_8562645,Salad,NaN,5.0,NaN,18/05/2023
278,TXN_3229409,Juice,NaN,3.0,NaN,15/04/2023
641,TXN_2962976,Juice,NaN,3.0,NaN,17/03/2023
738,TXN_8696094,Sandwich,NaN,4.0,NaN,14/05/2023
1761,TXN_3611851,NaN,4.0,NaN,NaN,09/02/2023
2289,TXN_7524977,UNKNOWN,4.0,NaN,NaN,09/12/2023
2796,TXN_9188692,Cake,NaN,3.0,NaN,01/12/2023
3203,TXN_4565754,Smoothie,NaN,4.0,NaN,06/10/2023
3224,TXN_6297232,Coffee,NaN,2.0,NaN,07/04/2023
3401,TXN_3251829,Tea,NaN,1.5,NaN,25/07/2023


## Handle Missing and Invalid Values

The dataset contains missing or invalid values represented as `UNKNOWN`, `ERROR`, or `NaN` (blank).  

- String-based invalid values will be standardized to `"Unknown"`.  
- Date columns will be assigned `NaN`.
> **Note:** Missing/invalid numeric columns are already in `NaN` due to a previous cleaning step.

This ensures consistency across the dataset for analysis.


In [220]:
# Define invalid string values to clean
invalid_strings = ['ERROR', 'UNKNOWN', '', None]


# Replace invalid or blank values in the columns with "Unknown"
menu_clean['item'] = menu_clean['item'].replace(invalid_strings, 'Unknown')

# Replace invalid values in 'transaction_date' with NaN
menu_clean['transaction_date'] = menu_clean['transaction_date'].replace(invalid_strings, pd.NA)


In [221]:
# Check
menu_clean['item'].value_counts(dropna=False)

item
Coffee      1291
Salad       1272
Cookie      1213
Tea         1207
Juice       1171
Cake        1139
Sandwich    1131
Smoothie    1096
Unknown      480
Name: count, dtype: int64

In [222]:
# Check
menu_clean['transaction_date'].value_counts(dropna=False)

transaction_date
<NA>          460
16/06/2023     40
06/02/2023     40
21/09/2023     39
21/07/2023     39
             ... 
24/11/2023     15
27/04/2023     15
22/07/2023     14
17/02/2023     14
11/03/2023     14
Name: count, Length: 366, dtype: int64

In [223]:
# Save the cleaned DataFrame to a new CSV file
menu_clean.to_csv("cafe_sales_clean.csv", index=False)
